### Importação de bibliotecas

In [89]:
# Importar bibliotecas

import pandas as pd
import numpy as np
import plotly.graph_objects as go

### Leitura e preparação dos dados da pesquisa de satisfação

In [90]:
# Ler dados da Pesquisa de Satisfação

dados = pd.read_excel('Dados_Satisfacao_Clientes.xlsx',index_col = 'ID_CLIENTE')

In [91]:
dados.head()

,SATISFACAO_GERAL,INFRA_1,INFRA_1_1,INFRA_1_2,PROF_1,PROF_1_1,PROD_1,PROD_1_1,NPS
ID_CLIENTE,,,,,,,,,
1,4.0,3.0,4.0,1.0,4.0,4.0,4.0,4.0,8.0
2,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,10.0
3,4.0,2.0,4.0,1.0,5.0,5.0,5.0,5.0,9.0
4,4.0,3.0,4.0,5.0,4.0,4.0,NaN,NaN,10.0
5,5.0,4.0,5.0,3.0,5.0,5.0,NaN,NaN,9.0


In [92]:
# Imputação de dados

for coluna in dados.columns.values:
    dados[coluna].fillna(dados[coluna].median(),inplace=True)

In [93]:
dados.head()

,SATISFACAO_GERAL,INFRA_1,INFRA_1_1,INFRA_1_2,PROF_1,PROF_1_1,PROD_1,PROD_1_1,NPS
ID_CLIENTE,,,,,,,,,
1,4.0,3.0,4.0,1.0,4.0,4.0,4.0,4.0,8.0
2,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,10.0
3,4.0,2.0,4.0,1.0,5.0,5.0,5.0,5.0,9.0
4,4.0,3.0,4.0,5.0,4.0,4.0,3.0,3.0,10.0
5,5.0,4.0,5.0,3.0,5.0,5.0,3.0,3.0,9.0


### Resultados da Pesquisa de Satisfação

In [94]:
# Resultado das perguntas de satisfação:
# Insatisfeito: 1 e 2
# Neutro: 3
# Satisfeito: 4 e 5

dados_ = dados[['SATISFACAO_GERAL', 'INFRA_1', 'INFRA_1_1', 'INFRA_1_2', 'PROF_1',
                'PROF_1_1', 'PROD_1', 'PROD_1_1']].replace(1,"Insatisfeito").replace(2,"Insatisfeito").replace(3,"Neutro").replace(4,"Satisfeito").replace(5,"Satisfeito")

In [95]:
# Resultado da pergunta de NPS (Net Promoter Score):
# Detrator: 1 a 6
# Neutro: 7 e 8
# Promotor: 9 e 10

dados_['NPS'] = ''
dados_.loc[dados['NPS'] <= 6, 'NPS'] = 'Detrator'
dados_.loc[(dados['NPS'] == 7) | (dados['NPS'] == 8) , 'NPS'] = 'Neutro'
dados_.loc[dados['NPS'] >= 9, 'NPS'] = 'Promotor'

In [96]:
dados_.head()

,SATISFACAO_GERAL,INFRA_1,INFRA_1_1,INFRA_1_2,PROF_1,PROF_1_1,PROD_1,PROD_1_1,NPS
ID_CLIENTE,,,,,,,,,
1,Satisfeito,Neutro,Satisfeito,Insatisfeito,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Neutro
2,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Promotor
3,Satisfeito,Insatisfeito,Satisfeito,Insatisfeito,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Promotor
4,Satisfeito,Neutro,Satisfeito,Satisfeito,Satisfeito,Satisfeito,Neutro,Neutro,Promotor
5,Satisfeito,Satisfeito,Satisfeito,Neutro,Satisfeito,Satisfeito,Neutro,Neutro,Promotor


In [97]:
# Compilação dos resultados para visualização

agrupado = pd.DataFrame(columns = ['Insatisfeito','Neutro','Satisfeito'])
for coluna in dados_.columns.values[0:-1]:
    agrupado = agrupado.append({'Insatisfeito': dados_[coluna].value_counts(normalize=True).sort_index().values[0],
                                'Neutro': dados_[coluna].value_counts(normalize=True).sort_index().values[1],
                                'Satisfeito':dados_[coluna].value_counts(normalize=True).sort_index().values[2]}, ignore_index=True)
agrupado.set_index(dados_.columns.values[0:-1])

,Insatisfeito,Neutro,Satisfeito
SATISFACAO_GERAL,0.175879,0.190955,0.633166
INFRA_1,0.246231,0.341709,0.412060
INFRA_1_1,0.226131,0.155779,0.618090
INFRA_1_2,0.165829,0.185930,0.648241
PROF_1,0.165829,0.175879,0.658291
PROF_1_1,0.180905,0.201005,0.618090
PROD_1,0.221106,0.376884,0.402010
PROD_1_1,0.246231,0.396985,0.356784


In [98]:
# Visualização dos resultados de satisfação

x=['Satisfação Geral', 'Infraestrutura', 'Banheiros', 'Salas', 'Profissionais', 'Atendimento', 'Produtos', 'Qualidade']
fig = go.Figure(go.Bar(x=x, y=list(agrupado.Insatisfeito.values), name='Insatisfeito',marker_color='red'))
fig.add_trace(go.Bar(x=x, y=list(agrupado.Neutro.values), name='Neutro',marker_color='grey'))
fig.add_trace(go.Bar(x=x, y=list(agrupado.Satisfeito.values), name='Satisfeito',marker_color='green'))

fig.update_layout(title = 'Pesquisa de Satisfação', barmode='stack')
fig.show()

In [99]:
# Visualização dos resultados de NPS

x=['NPS']
fig = go.Figure(go.Bar(x=x, y=[dados_['NPS'].value_counts(normalize=True).sort_index().values[0]], name='Detrator',marker_color='red'))
fig.add_trace(go.Bar(x=x, y=[dados_['NPS'].value_counts(normalize=True).sort_index().values[1]], name='Neutro',marker_color='grey'))
fig.add_trace(go.Bar(x=x, y=[dados_['NPS'].value_counts(normalize=True).sort_index().values[2]], name='Promotor',marker_color='green'))

fig.update_layout(title = 'NPS', barmode='stack')
fig.show()

In [100]:
# Nota do NPS

print('Net Promoter Score: ' + str(dados_['NPS'].value_counts().sort_index()[2] - dados_['NPS'].value_counts().sort_index()[0]))

Net Promoter Score: -17


### Modelo a ser analisado

![title](SEM.png)

### Factor Analysis (FA)
A Análise Fatorial explora conjuntos de dados para encontrar relações ocultas entre as variáveis. A análise reduz as variáveis observáveis a um número menor de variáveis não-observáveis (latentes), ou identifica grupos de variáveis intra-relacionadas. A análise ajuda na pesquisa de mercado, pois encontra relações ocultas entre gosto e preferência de cliente, por exemplo.

In [87]:
# Importar biblioteca

from factor_analyzer import FactorAnalyzer   

In [16]:
# Separar dados para analisar os constructos (variáveis latentes)

dados_constructo = dados[['INFRA_1', 'INFRA_1_1', 'INFRA_1_2', 'PROF_1',
                          'PROF_1_1', 'PROD_1', 'PROD_1_1']]

In [17]:
# Instanciar o objeto para realizar a análise fatorial

fa = FactorAnalyzer(n_factors=3,rotation='varimax')
fa.fit(dados_constructo)

FactorAnalyzer(bounds=(0.005, 1), impute='median', is_corr_matrix=False,
        method='minres', n_factors=3, rotation='varimax',
        rotation_kwargs={}, use_smc=True)

In [19]:
# Loadings são similares a coeficientes padronizados de regressões. Variáveis com valores maiores para um
# determinado fator podem ser interpretados como se explicassem uma maior proporção da variância daquele fator

pd.DataFrame(fa.loadings_,columns=['Fator 1', 'Fator 2', 'Fator 3'], index = dados_constructo.columns.values)

,Fator 1,Fator 2,Fator 3
INFRA_1,0.560365,0.280708,0.198115
INFRA_1_1,0.493704,0.202497,0.304355
INFRA_1_2,0.626366,0.220537,0.182370
PROF_1,0.614350,0.127866,0.492511
PROF_1_1,0.347326,0.187380,0.917965
PROD_1,0.426292,0.614036,0.205062
PROD_1_1,0.202869,0.971658,0.117900


#### Fator 1: Infraestrutura (INFRA_1, INFRA_1_1 e INFRA_1_2)
#### Fator 2: Produtos (PROD_1 e PROD_1_1)
#### Fator 3: Pofissionais (PROF_1 e PROF_1_1)

In [20]:
# Variânca dos fatores

pd.DataFrame(fa.get_factor_variance(),columns=['Infraestrutura', 'Produtos', 'Profissionais'], index = ['Variância','Variância Proporcional','Variância Acumulada'])

,Infraestrutura,Produtos,Profissionais
Variância,1.671028,1.541059,1.306317
Variância Proporcional,0.238718,0.220151,0.186617
Variância Acumulada,0.238718,0.458870,0.645486


### Structural Equation Modeling (SEM)

Modelagem de Equações Estruturais (SEM - *Structural Equation Modeling*), é um conjunto de técnicas estatísticas que permite, em uma única estrutura, estudar a relação entre todas as variáveis de um modelo. A SEM une técnicas multivariádas: uma série de equações lineares regridem, simultaneamente, cada variável independente na variável dependente, levando em consideração o impacto das outras variáveis na variável de interesse.

In [88]:
# Importar bibliotecas

import semopy
from semopy import Model
from semopy import Optimizer
from semopy.inspector import inspect
from semopy import gather_statistics

In [23]:
# Separar dados para criar o modelo de equações estruturais

dados_geral = dados[['SATISFACAO_GERAL', 'INFRA_1', 'INFRA_1_1', 'INFRA_1_2', 'PROF_1', 'PROF_1_1',
                     'PROD_1','PROD_1_1', 'NPS']]

In [80]:
# Definir o modelo de equações estruturais
# ~ : especifica a parte estrutural
# =~ : especifica a parte mensurável
# ~~ : especifica covariancia entre variáveis

modelo_geral = """ 
                   infraestrutura =~ INFRA_1 + INFRA_1_1 + INFRA_1_2
                   profissionais =~ PROF_1 + PROF_1_1
                   produtos =~ PROD_1 + PROD_1_1
                   SATISFACAO_GERAL ~ infraestrutura + profissionais + produtos
                   NPS ~ SATISFACAO_GERAL
                   
                   INFRA_1 ~~ INFRA_1_1
                   INFRA_1 ~~ INFRA_1_2
                   
                   PROD_1 ~~ PROD_1_1

                   PROF_1 ~~ PROF_1_1
             
                   INFRA_1, INFRA_1_1, INFRA_1_2, PROF_1, PROF_1_1, PROD_1, PROD_1_1, SATISFACAO_GERAL, NPS is ordinal
               """

# Instanciar o modelo

model = Model(modelo_geral)

In [81]:
# Carregar os dados para rodar o modelo

model.load_dataset(dados_geral)

In [82]:
# Rodar o otimizador para estimar os parâmetros do modelo

opt = Optimizer(model)
of = opt.optimize(objective='ULS')

In [83]:
# Estatísticas do modelo

stats = gather_statistics(opt)
print("DoF: " + str(model.calc_df()))
print("CFI: " + str(stats.cfi))
print("RMSEA: " + str(stats.rmsea))
print("NFI: " + str(stats.nfi))
print("GFI: " + str(stats.gfi))
print("Fit: " + str(of))

DoF: 18.0
CFI: 0.9952037991989747
RMSEA: 0.06569959394230834
NFI: 0.9897074601871796
GFI: 0.9897074601871796
Fit: 0.16775768944659544


In [84]:
# Resultados do Modelo

resultados = inspect(opt)
resultados.loc[(resultados['op'] == '~') & (resultados['P-value'] <= 0.05), ['lval', 'op', 'rval', 'Value']]

,lval,op,rval,Value
1,SATISFACAO_GERAL,~,infraestrutura,0.221046
2,SATISFACAO_GERAL,~,produtos,0.593239
3,SATISFACAO_GERAL,~,profissionais,0.347472
